In [188]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os,subprocess

In [2]:
pwd()


'C:\\Users\\aparashar'

In [3]:
%cd E:\\META\\ltrc\\ace-0.9.24

E:\META\ltrc\ace-0.9.24


In [54]:
def get_pos(pos,meaning):
    if "adj." in meaning:
        return "adj"
    elif "to " in meaning:
        return "verb"
    elif "adv. " in meaning:
        return "adv"
    elif pos != "":
        return pos
    else:
        return "noun"

In [55]:
# filtering words and meaning from the html pages
def get_word_meaning(results,d,ind):
    for i in results[1:]:
        try:
            hw = i.find("hw")
            pos = ""
            #print(hw)
            if hw:
                if "adj." in hw.next_sibling:
                    pos = "adj"
                elif "adv. " in hw.next_sibling:
                    pos = "adv"
            
            if hw and i.find("def"):

                defs = i.find_all("def")

                for j in defs[1:]:
                    t = j.previous_sibling
                    if t[0].isdigit():
                        t = t[3:]
                    colon_split = t.split(";")
                    for _ in colon_split:
                        comma_splt = _.split(",")
                        for k in comma_splt:
                            d["word"].append(hw.text.split(" ")[0])
                            d["meaning"].append(k.strip())
                            d["pos"].append(get_pos(pos,k.strip(' ')))
                t = defs[-1].contents[0]
                if t[0].isdigit():
                        t = t[3:]
                colon_split = t.split(";")
                for _ in colon_split:
                    comma_splt = _.split(",")
                    for k in comma_splt:
                        d["word"].append(hw.text.split(" ")[0])
                        d["meaning"].append(k.strip())
                        d["pos"].append(get_pos(pos,k.strip(' ')))

            elif hw and (i.find("def")==None):
                if i.find("i"):
                    i_s = i.find_all("i")
                    t = i_s[-1].next_sibling
                else:
                    t = hw.next_sibling
                    
                colon_split = t.split(";")
                for _ in colon_split:
                    comma_splt = _.split(",")
                    for k in comma_splt:
                        d["word"].append(hw.text.split(" ")[0])
                        d["meaning"].append(k.strip())
                        d["pos"].append(get_pos(pos,k.strip(' ')))
            
            shws = i.find_all("shw")
            for j in shws:
                pos = ""
                if "adj." in j.next_sibling:
                    pos = "adj"
                elif "adv. " in j.next_sibling:
                    pos = "adv"
                if "~" in j.contents[0]:
                    devas = j.find_all("deva")
                    for k in devas:
                        word = hw.text.split(" ")[0] +" "+ k.text
                        #print([word,j.next_sibling])
                        colon_split = j.next_sibling.split(";")
                        for _ in colon_split:
                            comma_splt = _.split(",")
                            for k in comma_splt:
                                d["word"].append(word)
                                d["meaning"].append(k.strip())
                                d["pos"].append(get_pos(pos,k.strip(' ')))

                elif "˚" in j.contents[0]:
                    deva = j.find("deva")
                    word = d["word"][-1] + "˚"+ deva.text
                    #print([word,j.next_sibling])
                    #d["word"].append(word)
                    if "[" in j.next_sibling:
                        while  "]" not in j.next_sibling:
                            j = j.next_sibling
                    colon_split = j.next_sibling.split(";")
                    for _ in colon_split:
                        comma_splt = _.split(",")
                        for k in comma_splt:
                            d["word"].append(word)
                            d["meaning"].append(k.strip())
                            d["pos"].append(get_pos(pos,k.strip(' ')))
                else:
                    devas = j.find_all("deva")
                    for k in devas:
                        word = k.text
                        #print([word,j.next_sibling])
                        if "[" in j.next_sibling:
                            while  "]" not in j.next_sibling:
                                j = j.next_sibling
                        #print(j.next_sibling)
                        colon_split = j.next_sibling.split(";")
                        for _ in colon_split:
                            comma_splt = _.split(",")
                            for k in comma_splt:
                                d["word"].append(word)
                                d["meaning"].append(k.strip())
                                d["pos"].append(get_pos(pos,k.strip(' ')))
        except:
            # returning which words on which pages threw exception
            print([ind,hw])
            continue
   

#print(d)

In [165]:
d = {"word":[],"meaning":[],"pos":[]}
pindex = 1
# total 1083 pages
for i in range(1084):
    URL = "https://dsal.uchicago.edu/cgi-bin/app/mcgregor_query.py?page=" + str(i)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="results_display")
    results  = results.find_all("div")
    while soup.find("sp"):
        soup.sp.extract()
    get_word_meaning(results,d,i)

[50, <hw><deva>अभी</deva> <tran>abhī</tran></hw>]
[61, <hw><deva>अवतार</deva> <tran>avă-tār</tran></hw>]
[95, <hw><deva>आवाज़</deva> <tran>āvāz</tran></hw>]
[98, <hw><deva>आसन्न</deva> <tran>ā-sann</tran></hw>]
[99, <hw><deva>आह</deva> <tran>āh</tran></hw>]
[105, <hw><deva>इनाम</deva> <tran>inām</tran></hw>]
[107, <hw><deva>इलायची</deva> <tran>ilāycī</tran></hw>]
[110, <hw><deva>ईमान</deva> <tran>īmān</tran></hw>]
[112, <hw><deva>उँगली</deva> <tran>uṁglī</tran></hw>]
[114, <hw><deva>उचाट</deva> <tran>ucāṭ</tran></hw>]
[123, <hw><deva>उदर</deva> <tran>udar</tran></hw>]
[131, <hw><deva>उम्मीद</deva> <tran>ummīd</tran></hw>]
[136, <hw><deva>ऊँचा</deva> <tran>ūṁcā</tran></hw>]
[136, <hw><deva>ऊँट</deva> <tran>ūṁṭ</tran></hw>]
[142, <hw><deva>एतबार</deva> <tran>etbār</tran></hw>]
[147, <hw><deva>ओर</deva> <tran>or</tran></hw>]
[151, <hw><deva>औसान</deva> <tran>ausān</tran></hw>]
[162, <hw><deva>क़तरा</deva> <tran>qatrā</tran></hw>]
[166, <hw><deva>कफ़</deva> <tran>kaf</tran></hw>]
[172, <hw><d

In [166]:
df =pd.DataFrame.from_dict(d)

In [168]:
# dropping improper meaning rows
l = ['[S.]',']','.','˚]']
df = df[df['meaning'].str.strip().astype(bool)]
indexNames = df[df['meaning'].isin(l)].index
df = df.drop(indexNames).reset_index()
#df[300:320]

In [173]:
df.loc[df.meaning.apply(lambda row: "= next" in row),"meaning"] = df["meaning"].shift(-1)

In [174]:
df[df.meaning.apply(lambda row: "= next" in row)]

,index,word,meaning,pos


In [175]:
# removing unwanted characters from the text
unw = ['. —','m. ','adv. ','id.','adj. ','hw.','f. ',' (=','&c.','= ~','(= ~',')','Brbh. =','v.t.','Brbh.','Av.','v.i.','inv.','U.','? ← Fr.]','&','.','[',']','-','=',' –']
def clean_set(m):
    ret = m
    ret = re.sub("[\(\[].*?[\)\]]", "", m)
    ret = ret.split('(')[0]
    for i in unw:
        ret = ret.replace(i,'')
    
    return ret.strip()
    

In [176]:
# last check for 'verb' pos
def handle_1(row):
    
    if row['meaning'][:3] == 'to ':
        return 'verb'
    else:
        return row['pos']
# removing 'ना' suffix if the pos is a verb
def handle_2(row):
    row["word"] = re.sub("[\(\[].*?[\)\]]", "", row["word"])
    row['word'] = row['word'].strip(' ')
    if row['pos'] == 'verb' and row['word'][-2:] == 'ना':
        l = len(row['word'])
        return row['word'][:l-2]
    else:
        return row['word']

# removing articles from the beginning of meaning
def handle_3(row):
    ret = row['meaning']
    space_splt = ret.split(' ')
    arts = ['a','the','an']
    if space_splt[0] in arts:
        ret = ' '.join(space_splt[1:])
    if space_splt[0] == 'to' and row['pos'] == 'verb':
        ret = ' '.join(space_splt[1:])
    return ret


def get_sentence(word,pos):
    if pos == 'verb' :
        return 'Please {}.'.format(word)
    elif pos == 'adj' :
        return 'This is {}.'.format(word)
    elif pos == 'noun' :
        return 'This is the {}.'.format(word)
    if pos == 'adv': 
        return 'He wants to do it {}.'.format(word)
    else:
        return word

In [177]:
df['meaning'] = df.meaning.apply(lambda row: clean_set(row))
df['pos'] = df.apply(lambda row: handle_1(row),axis=1)
df['word'] = df.apply(lambda row: handle_2(row),axis=1)
df['meaning'] = df.apply(lambda row: handle_3(row),axis=1)
#df.drop(df.index[df['word'] == None], inplace=True)

In [178]:
df

,index,word,meaning,pos
0,1,अ,first vowel of the Devanāgarī syllabary,noun
1,3,अकार,sound /ə/,noun
2,4,अकार,letter,noun
3,6,अकार˚आदि क्रम,alphabetical order,noun
4,7,अंक,number,noun
...,...,...,...,...
149583,200929,ई,,adj
149584,200931,ह्री,poet shame,noun
149585,200932,ह्री,modesty,noun
149586,200934,ह्वाँ,,adv


In [180]:
def handle_4(row):
    ret = row['word']
    ret = ret.strip(' ')
    return ret.replace(' ','+')

In [182]:
# filtering single worded meanings from the df dataframe to sdf dataframe
sdf = df[df.meaning.apply(lambda row: row.find(' ') == -1)]
sdf['word'] = sdf.apply(lambda row:handle_4(row),axis =1)
sdf = sdf[sdf.meaning != '']
sdf[sdf['pos']=='verb']

C:\Users\aparashar\AppData\Local\Temp\ipykernel_8760\2803608685.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['word'] = sdf.apply(lambda row:handle_4(row),axis =1)


,index,word,meaning,pos
26,30,अंक+दे,embrace,verb
27,32,अंक+भर,embrace,verb
28,34,अंक+लगा,embrace,verb
57,78,अँकड़ा,parch,verb
58,79,अँकड़ा,dry,verb
...,...,...,...,...
149434,200723,होली+की,squander,verb
149495,200810,होशियार+कर,apprise,verb
149496,200811,होशियार+कर,forewarn,verb
149507,200826,हौंक,pant,verb


In [183]:
word_dict = {}
meaning_dict = {}
# adding count at the end of the words and meanings

def count_word(row):
    word = row['word']
    if word in word_dict:
        word_dict[word] +=1
        return word+'_'+ str(word_dict[word])
    else:
        word_dict[word] =1
        return word+'_'+ str(word_dict[word])
def count_meaning(row):
    meaning = row['meaning']
    if meaning in meaning_dict:
        meaning_dict[meaning] +=1
        return meaning+'_'+ str(meaning_dict[meaning])
    else:
        meaning_dict[meaning] =1
        return meaning+'_'+ str(meaning_dict[meaning])
        

In [184]:

sdf['word'] = sdf.apply(lambda row:count_word(row),axis =1)
sdf['meaning'] = sdf.apply(lambda row:count_meaning(row),axis =1)

sdf

,index,word,meaning,pos
2,4,अकार_1,letter_1,noun
4,7,अंक_1,number_1,noun
5,8,अंक_2,figure_1,noun
6,9,अंक_3,mark_1,noun
7,10,अंक_4,spot_1,noun
...,...,...,...,...
149574,200917,ह्रस्वता_1,shortness_2,noun
149577,200920,हृदिनी_1,lightning_14,noun
149580,200924,ह्रास_1,declining_3,noun
149582,200927,ह्रासोन्मुख_1,declining_4,noun


In [185]:
sdf.reset_index()

,level_0,index,word,meaning,pos
0,2,4,अकार_1,letter_1,noun
1,4,7,अंक_1,number_1,noun
2,5,8,अंक_2,figure_1,noun
3,6,9,अंक_3,mark_1,noun
4,7,10,अंक_4,spot_1,noun
...,...,...,...,...,...
81575,149574,200917,ह्रस्वता_1,shortness_2,noun
81576,149577,200920,हृदिनी_1,lightning_14,noun
81577,149580,200924,ह्रास_1,declining_3,noun
81578,149582,200927,ह्रासोन्मुख_1,declining_4,noun


In [186]:
sdf = sdf.drop(columns=["index"])
sdf.to_csv("1-1083.txt",sep="\t")

In [187]:
sdf



,word,meaning,pos
2,अकार_1,letter_1,noun
4,अंक_1,number_1,noun
5,अंक_2,figure_1,noun
6,अंक_3,mark_1,noun
7,अंक_4,spot_1,noun
...,...,...,...
149574,ह्रस्वता_1,shortness_2,noun
149577,हृदिनी_1,lightning_14,noun
149580,ह्रास_1,declining_3,noun
149582,ह्रासोन्मुख_1,declining_4,noun
